# Explore and cluster the neighborhoods in Toronto Part - 1
### Scrape Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


# Option A
### We can use BeautifulSoup


# Option B 
### Use python pandas 

### visit the web page "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
##### Explore the web page
##### It looks not very complicated so python pandas can be used to read & load the data

In [48]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

print('Libraries imported.')

Libraries imported.


In [49]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikipage = pd.read_html(url)



# Explore Data and its struture

In [50]:
type(wikipage)

list

In [51]:
len(wikipage)

3

### Check the every element of list 'wikipage[0], 'wikipage[1], 'wikipage[2] one by one

In [52]:
wikipage[0].head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [53]:
wikipage[1].head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,NaN,Canadian postal codes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL,NS,PE,NB,QC,ON,MB,SK,AB,...,L,M,N,P,R,S,T,V,X,Y
2,NL,NS,PE,NB,QC,ON,MB,SK,AB,BC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,B,C,E,G,H,J,K,L,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NL,NS,PE,NB,QC,ON,MB,SK,AB,BC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
wikipage[2].head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NL,NS,PE,NB,QC,ON,MB,SK,AB,BC,NU/NT,YT,NaN,NaN,NaN,NaN,NaN,NaN
1,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


# Data Selection

### Now it is clear wikipage is a nested list structure and our data is in "wikipage[0]"
#### Create a data frame Totonto_df with wikipage[0]

In [55]:
Totonto_df = wikipage[0]
Totonto_df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### First row is holds the name of columns 
#### so we will make it the column name of daraframe  and delete the first row

In [56]:
Totonto_df[0:1].values.tolist()

[['Postcode', 'Borough', 'Neighbourhood']]

In [57]:
Totonto_df.columns=['Postcode','Borough','Neighbourhood']

#### Remove first row it holds the name of column and reset the index

In [58]:
#Totonto_df = Totonto_df.iloc[1:]
Totonto_df.drop(Totonto_df.index[:1], inplace=True)
Totonto_df.reset_index(drop=True, inplace=True)
Totonto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [59]:
Totonto_df.shape

(289, 3)

# Data Preparation


### Target 1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [60]:
Totonto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Target 2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [61]:
Totonto_df.replace({'Borough': 'Not assigned' }, np.nan, inplace = True)
Totonto_df.head()
# simply drop whole row with NaN in "price" column
Totonto_df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped two rows
Totonto_df.reset_index(drop=True, inplace=True)
Totonto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [62]:
Totonto_df.shape

(212, 3)

### Target 3. More than one neighborhood can exist in one postal code area. 
#### For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [63]:
# Groupby and join can be used for the purpose
Totonto_df = Totonto_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
Totonto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [64]:
Totonto_df.shape

(103, 3)

### Target 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [65]:
#Totonto_df[ (Totonto_df.Neighbourhood=='Not assigned')]['Neighbourhood'] = [Borough]
Totonto_df['Neighbourhood'] = np.where(Totonto_df['Neighbourhood'] == 'Not assigned', Totonto_df['Borough'], Totonto_df['Neighbourhood'])
Totonto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Target 5. Use the .shape method to print the number of rows of your dataframe.

In [66]:
Totonto_df.shape

(103, 3)